## ============   Decision Trees    ===============

##### Notes: https://www.cse.iitd.ac.in/~parags/teaching/2023/col774/supp_material/dtrees.pdf

In [1]:
depth_limit          = 8    # grow the tree only upto this depth
use_one_hot_encoding = 0    # i do not understand, why one hot encoding can be useful here
                            # it makes more sense with neural network; you are right
                            # using one hot is incresing the dimension unneccarily and I cannot grow tree completely or use post pruning
                            # we remove the original categorical attribute after applying it on the attribute
use_post_pruning     = 1    # = 1 avoids overfitting

# default with depth: 5           = 0.675
# + use_post_pruning              = 0.675
# + use_one_hot_encoding          ~ 0.5

# default with depth: 8           = 0.65
# + use_post_pruning              = 0.677   (0.68 for valid)      ========  BEST
# + use_one_hot_encoding          ~ 0.5

# need_label_encoding = ['team','opp', 'day_match']
# depth = 50   (without use_post_pruning)
# + use_one_hot_encoding          = 0.62   (0.99 on train)

assert(depth_limit          >= 0)
assert(use_one_hot_encoding == 0   or    use_one_hot_encoding == 1)
assert(use_post_pruning     == 0   or    use_post_pruning     == 1)  

#need_label_encoding = ['team','host','opp','month', 'day_match']
#need_label_encoding = ['team','host','opp', 'day_match']
need_label_encoding = ['team','opp', 'day_match']
#need_label_encoding = ['team', 'day_match']

### Loading Data

In [2]:
import pandas as pd
df = pd.read_csv('dataset_cricket_match/train.csv')
df = df.drop("Unnamed: 0", axis = 1)
df.head(5)
# fow: fall of wicket;   rpo: run per over or run rate

,team,opp,host,year,month,toss,day_match,bat_first,format,fow,score,rpo,result
0,australia,south_africa,sri_lanka,2012,sep,1,0,0,1,5,146,7.30,1
1,india,australia,india,2020,jan,0,0,1,0,6,340,6.80,1
2,canada,scotland,scotland,2009,jul,1,1,0,0,4,286,5.72,1
3,australia,england,australia,1987,jan,1,1,1,0,6,225,4.50,1
4,new_zealand,pakistan,uae,2009,nov,0,0,0,1,5,153,7.65,0


### Convert Categorical Data to One Hot Encoding

In [3]:
def convert_df_to_one_hot(df):
    for attr_name in need_label_encoding:
        unique_attr = df[attr_name].unique()
        for attr_val in unique_attr:
            new_attr_name = attr_name + "_" + str(attr_val)
            df[new_attr_name] = 0
            df.loc[df[attr_name] == attr_val, new_attr_name] = 1
        df = df.drop(attr_name, axis = 1) # drops columns
    return df
    
if use_one_hot_encoding == 1:
   df = convert_df_to_one_hot(df) 
df.head()

,team,opp,host,year,month,toss,day_match,bat_first,format,fow,score,rpo,result
0,australia,south_africa,sri_lanka,2012,sep,1,0,0,1,5,146,7.30,1
1,india,australia,india,2020,jan,0,0,1,0,6,340,6.80,1
2,canada,scotland,scotland,2009,jul,1,1,0,0,4,286,5.72,1
3,australia,england,australia,1987,jan,1,1,1,0,6,225,4.50,1
4,new_zealand,pakistan,uae,2009,nov,0,0,0,1,5,153,7.65,0


### STATS

In [4]:
print ("number of rows         : ", df.shape[0])
print ("number of features     : ", df.shape[1]-1)  # excluding the class labels
print ("number of class labels : ", 2)
#df.info()

number of rows         :  7827
number of features     :  12
number of class labels :  2


### Identify Attributes as Categorical or Continuous

In [5]:
attr_list = df.axes[1].tolist()
attr_list.remove('result')
#print (attr_list)
cont_attr = ['fow', 'score', 'rpo']
#print (cont_attr)

# only fow, score, rpo are continuous = 0; rest are categorical = 0

### Compute Entropy

In [6]:
import math
def getEntropyDataframe(df):
    cnt_dict =  df["result"].value_counts().to_dict()
    cnt_list =  [int(cnt_dict[0]), int(cnt_dict[1])]
    return getEntropyList(cnt_list)

def getEntropyList(cnt_list):
    if cnt_list[0] == 0 or cnt_list[1] == 0:     # 0 error when 0 entropy; all data items have the same class label 
        return 0
    total    =  sum(cnt_list)
    p0       =  cnt_list[0]/total
    p1       =  cnt_list[1]/total
    entropy  =  -1*(p0*math.log(p0,2) + p1*math.log(p1,2))
    return entropy

print (getEntropyDataframe(df))

0.9999669243084066


In [7]:
import math
def getEntropyPostCatAttributeSplit(df, attribute):
    assert (attribute not in cont_attr)
    entropy         = 0
    dict_attr_count = df.value_counts([attribute,'result']).to_dict()
    total           = sum(dict_attr_count.values())
    unique_attr     = df[attribute].unique()
    for attr in unique_attr:
        if (attr,1) not in dict_attr_count:
            dict_attr_count[(attr,1)] = 0
        if (attr,0) not in dict_attr_count:
            dict_attr_count[(attr,0)] = 0
        attr_cnt_list  = [dict_attr_count[(attr,0)], dict_attr_count[(attr,1)]] 
        prob           =  sum(attr_cnt_list)/total
        entropy       += getEntropyList(attr_cnt_list)*prob
    return entropy

def getEntropyPostContAttributeSplit(df, attribute):
    assert (attribute in cont_attr)
    entropy         = 0
    mean_attr       = df.loc[:, attribute].mean()
    low_val_dict    = df[df[attribute] <=   mean_attr].value_counts('result').to_dict()
    high_val_dict   = df[df[attribute]  >   mean_attr].value_counts('result').to_dict()
    total           = df.shape[0]
    
    dict_attr_count = {}
    if 1 not in low_val_dict:
        low_val_dict[1] = 0
    if 0 not in low_val_dict:
        low_val_dict[0] = 0
    attr_cnt_list  = [low_val_dict[0], low_val_dict[1]] 
    prob           =  sum(attr_cnt_list)/total
    low_val_etpy   = getEntropyList(attr_cnt_list)*prob

    dict_attr_count = {}
    if 1 not in high_val_dict:
        high_val_dict[1] = 0
    if 0 not in high_val_dict:
        high_val_dict[0] = 0
    attr_cnt_list  = [high_val_dict[0], high_val_dict[1]] 
    prob           =  sum(attr_cnt_list)/total
    high_val_etpy  = getEntropyList(attr_cnt_list)*prob
    
    return low_val_etpy + high_val_etpy

#print (getEntropyPostCatAttributeSplit(df, 'team'))
print (getEntropyPostContAttributeSplit(df, 'fow'))

0.9999665065933905


### Choose Best Attribute & Split Dataframe

In [8]:
def chooseBestAttr(df, attr_list):   # attr_list: list of attributes to choose from
    assert (len(attr_list) > 0)
    min_entropy = 1
    best_attr   = ""
    for attr_name in attr_list:       # choose the best attribute based on minimum entropy
        if attr_name not in cont_attr:
            entropy = getEntropyPostCatAttributeSplit(df, attr_name)      # categorical attribute
        else:
            entropy = getEntropyPostContAttributeSplit(df, attr_name)     # continuous attribute
            
        #print ("entropy of ", attr_name, " is ", entropy)
        if entropy <= min_entropy:
            best_attr = attr_name
    return best_attr

def splitMinEntropy(df, attr_list, child_df_list, selected_attr, attribute_value_list, mean_attr_val):
    selected_attr[0]   = chooseBestAttr(df, attr_list)
    #print ("best attribute to split on: ", selected_attr[0])
    attr_name = selected_attr[0]
    
    if attr_name not in cont_attr:
        unique_attr = df[attr_name].unique()
        for attr_val in unique_attr:
            childDf = df.loc[df[attr_name] == attr_val]
            child_df_list.append(childDf)
            attribute_value_list.append(attr_val)
    else:
        mean_attr     = df.loc[:, attr_name].mean()
        low_val_df    = df[df[attr_name] <=   mean_attr]
        high_val_df   = df[df[attr_name]  >   mean_attr]
        child_df_list.append(low_val_df)
        child_df_list.append(high_val_df)
        mean_attr_val[0] = mean_attr
    #attr_list.remove(best_attr)             # no need to split on this attribute again since already utilized 

attr_list = df.axes[1].tolist()
attr_list.remove('result')
print (attr_list)
child_df_list        = []
attribute_value_list = []
selected_attr        = [0]
mean_attr_val        = [0]
splitMinEntropy(df, attr_list, child_df_list, selected_attr, attribute_value_list, mean_attr_val)

['team', 'opp', 'host', 'year', 'month', 'toss', 'day_match', 'bat_first', 'format', 'fow', 'score', 'rpo']


### Tree Node

In [9]:
total_num_nodes = [0]
class treeNode:
    attribute_split  = ""    # attribute that we choose this node to split into
    attribute_value  = ""    # value corresponding to it as child node   (categorical type)
    mean_value       = -1    # mean corresponding  to it as parenet node (continuous  type)
    isLeaf           = 0
    childNodeList    = []    # list of all child nodes of type treeNode (corresponding to attribute_value)
                             # for continuous type, childNodeList[0] corresponds to <= mean_value and childNodeList[0] for rest
    majorityClass    = -1    # takes value 0 or 1 only when encountered
    depth            = 0     # root has depth = 0
    isPruningLeaf    = 0     # = 1 for internal nodes as well; when doing post pruning
    

### Create Decision Tree

In [10]:
def splitAndUpdatePara(root, df, attr_list):
    #print ("\nnumber of data points at the node: ", df.shape[0], " depth: ", root.depth)
    lable_cnt_dict = df.value_counts('result').to_dict()   # assign the label based on majorityClass
    if 0 not in lable_cnt_dict:
        root.majorityClass = 1
        root.isLeaf = 1
        return
    if 1 not in lable_cnt_dict:
        root.majorityClass = 0
        root.isLeaf = 1
        return

    if lable_cnt_dict[0] > lable_cnt_dict[1]:
        root.majorityClass = 0
    else:
        root.majorityClass = 1
            
    if root.depth >= depth_limit  or  len(df.value_counts('result').to_dict()) <= 1  or  len(attr_list) == 0:
        #print ("probabilistic decision taken at leaf node; num wrong pred: ", min(lable_cnt_dict.values()))
        total_wrong_decision[0] += min(lable_cnt_dict.values())
        root.isLeaf = 1
        return
        
    child_df_list        = []
    attribute_value_list = []
    mean_attr_val        = [-1]
    selected_attr        = [-1]  # contains only one attribute # declaring as list since we are passing it inside method
    splitMinEntropy(df, attr_list, child_df_list, selected_attr, attribute_value_list, mean_attr_val)

    if selected_attr[0] in cont_attr:                      # attribute type: continuous
        root.mean_value      = mean_attr_val[0]
        #print ("mean: ", root.mean_value)
    root.attribute_split = selected_attr[0]
    new_attr_list = attr_list.copy()
    new_attr_list.remove(root.attribute_split)             # no need to split on this attribute again since already utilized

    
    #print ("number of children: ", len(child_df_list), " at depth: ", root.depth)
    index = 0
    for child_df in child_df_list:
        global childNode
        childNode = treeNode()
        childNode.childNodeList = []
        childNode.depth         = root.depth + 1
        childNode.isPruningLeaf = 0
        total_num_nodes[0]     += 1
        (root.childNodeList).append(childNode)
        if selected_attr[0] not in cont_attr:              # attribute type: categorical
            childNode.attribute_value = attribute_value_list[index]
        splitAndUpdatePara(childNode, child_df, new_attr_list)
        index += 1

def createTree(root):
    attr_list = df.axes[1].tolist()
    attr_list.remove('result')
    #print (attr_list)
    splitAndUpdatePara(root, df, attr_list)

total_wrong_decision = [0]
root = treeNode()
root.childNodeList = []
root.depth         = 0
root.isPruningLeaf = 0
total_num_nodes[0] = 1 
createTree(root)
print ("total number of nodes in the tree: ", total_num_nodes[0])
print ("total number of incorrect decision taken by DT on the training set: ", total_wrong_decision[0])

total number of nodes in the tree:  1113
total number of incorrect decision taken by DT on the training set:  2154


### Predict on Dataframe

In [11]:
def predictTree(df, root, lable_0_indexes, lable_1_indexes):
    if df.empty:
        return

    isPruningLeaf = 0
    if use_post_pruning == 1 and root.isPruningLeaf == 1:
        isPruningLeaf = 1
    
    if root.isLeaf == 1 or isPruningLeaf == 1:
        indexes         =  list(df.index.values)
        #print ("encountered leaf node at depth: ", root.depth, " and predicted ", len(indexes), " labels")
        if root.majorityClass == 1:
            lable_1_indexes += indexes
        else:
            lable_0_indexes += indexes
        return

    attr_name = root.attribute_split
    if attr_name in cont_attr:                     # attribute type: continuous
        left_df  = df.loc[df[attr_name] <= root.mean_value]
        right_df = df.loc[df[attr_name]  > root.mean_value]
        if len(root.childNodeList) >= 1:
            predictTree(left_df,  root.childNodeList[0], lable_0_indexes, lable_1_indexes)        # there is a boundary case here; please take care later =====
        if len(root.childNodeList) >= 2:
            predictTree(right_df, root.childNodeList[1], lable_0_indexes, lable_1_indexes)
        return

    list_attr_value = []
    if attr_name not in cont_attr:                 # attribute type: categorical
        for childNode in root.childNodeList:
            attr_value = childNode.attribute_value
            child_df   = df.loc[df[attr_name] == attr_value]
            #print ("child_df : ",  child_df.shape[0])
            list_attr_value.append(attr_value)
            predictTree(child_df, childNode, lable_0_indexes, lable_1_indexes)

    # if a new attribute value, then assign then consider the node as leaf node and assign the majority class label
    remaining_df    = df.loc[~df[attr_name].isin(list_attr_value)].copy()
    if remaining_df.empty:
        return
        
    indexes         =  list(remaining_df.index.values)
    #print ("encountered new attribute value for attribute: ", attr_name)
    if root.majorityClass == 1:
        lable_1_indexes += indexes
    else:
        lable_0_indexes += indexes

### Loading & Predicting Training, Validation, Test Dataset

In [12]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

train_df = pd.read_csv('dataset_cricket_match/train.csv')
valid_df = pd.read_csv('dataset_cricket_match/val.csv')
test_df  = pd.read_csv('dataset_cricket_match/test.csv')
df_list  = [train_df, valid_df, test_df] 

index = 0
for df in df_list:
    df = df.drop("Unnamed: 0", axis = 1)
    if use_one_hot_encoding == 1:           # converting categorical attributes to one hot encoded vector
        df = convert_df_to_one_hot(df) 
    print ("number of rows         : ", df.shape[0])
    df_list[index] = df
    index += 1

number of rows         :  7827
number of rows         :  870
number of rows         :  967


In [13]:
def getAccuracy(df, enable_prints):
    lable_0_indexes = []
    lable_1_indexes = []
    predictTree(df, root, lable_0_indexes, lable_1_indexes)
    
    class_0_df     = pd.DataFrame(index=lable_0_indexes, columns=['predicted_result'])
    class_1_df     = pd.DataFrame(index=lable_1_indexes, columns=['predicted_result'])
    class_0_df["predicted_result"] = 0
    class_1_df["predicted_result"] = 1
    if class_0_df.empty:
        full_df = class_1_df
    elif class_1_df.empty:
        full_df = class_0_df
    else:
        full_df = pd.concat([class_0_df, class_1_df])
    pred_df = df.merge(full_df, how = "left", left_index=True, right_index=True)


    y_true = pred_df['result'].tolist()
    y_pred = pred_df['predicted_result'].tolist()
    classes = [0,1]
    if enable_prints == 1:
        print("PR Report         : \n", classification_report(y_true, y_pred, labels=classes, zero_division=0))
        print("Confusion Matrix  : \n", confusion_matrix(y_true, y_pred))
        #print("\nAccuracy        : ", accuracy_score(y_true, y_pred))
    return accuracy_score(y_true, y_pred)

for df in df_list:
    print ("\naccuracy: ", getAccuracy(df,0))


accuracy:  0.7247987734764277

accuracy:  0.6344827586206897

accuracy:  0.6763185108583247


### Post-Pruning Decision Tree

In [14]:
if use_post_pruning == 1:
    df = df_list[1]   # validation data set
    
    def runDFS(root, current_best_accuracy):  
        for childNode in root.childNodeList:
            runDFS(childNode, current_best_accuracy)
            
        root.isPruningLeaf = 1
        accuracy = getAccuracy(df,0)
        if current_best_accuracy[0] < accuracy:
            print ("obtained better validation test accuracy: ", accuracy)
            current_best_accuracy[0] = accuracy
        elif current_best_accuracy[0] > accuracy:
            root.isPruningLeaf = 0
    # note that visited vector is not required for dfs here; since it is a simple k-ary tree
    
    current_best_accuracy = [0]
    current_best_accuracy[0] = getAccuracy(df,0)
    runDFS(root, current_best_accuracy)

print ("\ntrain      accuracy: ", getAccuracy(df_list[0],0))
print ("\nvalidation accuracy: ", getAccuracy(df_list[1],0))
print ("\ntest       accuracy: ", getAccuracy(df_list[2],0))

obtained better validation test accuracy:  0.635632183908046
obtained better validation test accuracy:  0.6367816091954023
obtained better validation test accuracy:  0.6379310344827587
obtained better validation test accuracy:  0.6402298850574712
obtained better validation test accuracy:  0.6413793103448275
obtained better validation test accuracy:  0.6436781609195402
obtained better validation test accuracy:  0.6459770114942529
obtained better validation test accuracy:  0.6505747126436782
obtained better validation test accuracy:  0.6517241379310345
obtained better validation test accuracy:  0.6563218390804598
obtained better validation test accuracy:  0.6586206896551724
obtained better validation test accuracy:  0.6620689655172414
obtained better validation test accuracy:  0.6632183908045977
obtained better validation test accuracy:  0.6655172413793103
obtained better validation test accuracy:  0.6666666666666666
obtained better validation test accuracy:  0.6689655172413793
obtained 

##### JUNK (Entropy Example)

In [15]:
import math

entropy_1     = -1*(1/5*math.log(1/5,2) +  4/5*math.log(4/5,2))         # base 2
entropy_2     = -1*(8/20*math.log(8/20,2) +  12/20*math.log(12/20,2))   # base 2
entropy_full  =  1/3*entropy_1 + 2/3*entropy_2

print ("entropy for [8|2]  :",  entropy_1)
print ("entropy for [12|8] :",  entropy_2)
print ("total entropy      :",  entropy_full)

entropy for [8|2]  : 0.7219280948873623
entropy for [12|8] : 0.9709505944546686
total entropy      : 0.8879430945988998
